## Movie Review Sentiment Classification

### Following instructions from Ch.8 of "Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn and TensorFlow2" Third Edition by Sebastian Raschka & Vahid Mirjalili

### Using dataset provided by Stanford, at : https://ai.stanford.edu/~amaas/data/sentiment/

### Carter Roberts
### For Loyola University New Orleans
### March 17, 2025

In [1]:
# some of the notes are just for me and my forward-going comprehension, 
# but you should read them just in case
import pyprind as pyp
import pandas as pd
import os

# this will need to be adjusted to what your path for aclImdb dataset is
basepath = 'C:\\Users\\Carter\\Documents\\Github\\maLearningMovieClassification\\aclImdb'

# establishing diction for columns
labels = {'pos': 1, 'neg': 0}
# 50,000 step progress bar & empty dataframe
pbar = pyp.ProgBar(50000)
df = pd.DataFrame()
# sift through all of those two folders with those two review results
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        # fix up a variable 'path' for next loop
        path = os.path.join(basepath, s, l)
        # use 'path' to loop through all files in the current directory within 'test' or 'train'
        for file in sorted(os.listdir(path)):
            # 'infile' now defines the current total path which was assembled above
            with open(os.path.join(path, file), 'r', encoding = 'utf-8') as infile:
                # read, from the file, the written data, into these holders, 'txt'
                txt = infile.read()
                # keep adding these to the previously-empty dataframe
                df = df.append([[txt, labels[1]]], ignore_index = True)
                # iterate a step of the progress bar
                pbar.update()
# define the columns of this newly-filled dataframe
df.columns = ['review', 'sentiment']

AttributeError: 'DataFrame' object has no attribute 'append'